# Experiment 10 - Implementing Non Parametric Locally Weighted Regression on Non-linear Data

## Theory

### Parametric vs Non-Parametric Learning Algorithms
Parametric — In a Parametric Algorithm, we have a fixed set of parameters such as theta that we try to find(the optimal value) while training the data. After we have found the optimal values for these parameters, we can put the data aside or erase it from the computer and just use the model with parameters to make predictions. Remember, the model is just a function.

Non-Parametric — In a Non-Parametric Algorithm, you always have to keep the data and the parameters in your computer memory to make predictions. And that’s why this type of algorithm may not be great if you have a really really massive dataset.

### Need for NPLW Regression
We specifically apply this regression technique when the data to fit is non-linear. In Linear Regression we would fit a straight line to this data but that won’t work here because the data is non-linear and our predictions would end up having large errors. We need to fit a curved line so that our error is minimized.

### How NPLW Regression Works
In Locally weighted linear regression, we give the model the `x` where we want to make the prediction, then the model gives all the `x(i)`’s around that `x` a higher weight close to one, and the rest of `x(i)`’s get a lower weight close to zero and then tries to fit a straight line to that weighted `x(i)`’s data.

This means that if want to make a prediction for the green point on the x-axis (see figure below), the model gives higher weight to the input data i.e. `x(i)`’s near or around the circle above the green point and all else `x(i)` get a weight close to zero, which results in the model fitting a straight line only to the data which is near or close to the circle. The same goes for the purple, yellow, and grey points on the x-axis.

![](./assets/NPLWR-graph.webp)

### Important Formulae
In Linear regression, we had the following loss function 

![](./assets/linear-regression-loss-function.webp)

The modified loss for locally weighted regression 

![](./assets/modified-loss-function.webp)

`w(i)` (the weight for the ith training example) is the only modification.

where the weighting function is, 

![](./assets/weighting-function.webp)

`x` is the point where we want to make the prediction. `x(i)` is the `ith` training example.

The value of this function is always between 0 and 1.

So, if we look at the function, we see that

    If |x(i)-x| is small, w(i) is close to 1.
    If |x(i)-x| is large, w(i) is close to 0.

The `x(i)`’s which are far from `x` get `w(i)` close to zero and the ones which are close to `x`, get `w(i)` close to 1.

### Calculating Error
In the loss function, it translates to error terms for the `x(i)`’s which are far from `x` being multiplied by almost zero and for the `x(i)`’s which are close to `x` get multiplied by almost 1. In short, it only sums over the error terms for the `x(i)`’s which are close to `x`.

### Algorithm
Actually, there exists a closed-form solution for this algorithm which means that we do not have to train the model, we can directly calculate the parameter theta using the following formula.

![](./assets/closed-form-solution-for-theta.webp)

# Code

## Importing Libraries

In [75]:
import numpy as np
import matplotlib.pyplot as plt
import warnings

%matplotlib inline

warnings.filterwarnings('ignore')

## Generating Non-linear Data
A good way to create a non-linear dataset is to mix sines with different phases. The dataset we will work with in this experiment is created with the following Python script and exported to a CSV file.

Create a variable to set the number of samples to create.

In [76]:
numberSamples = 500

Now, creating a lambda function that takes the input and convolves it as a mixture of two `cos` functions.

In [77]:
deLinearise = lambda X: np.cos(1.5 * np.pi * X) + np.cos(5 * np.pi * X)

Now, creating `X` and `y` using the above function and `np.random.rand`.

In [78]:
X = np.sort(np.random.rand(numberSamples)) * 2
y = deLinearise(X) + np.random.randn(numberSamples) * 0.1

Here's what `X` looks like.

In [79]:
X, X.shape

(array([1.79952149e-04, 1.46677833e-02, 1.50654864e-02, 1.63207625e-02,
        1.81878382e-02, 2.47717645e-02, 2.85435622e-02, 2.97563727e-02,
        3.66570674e-02, 4.04090469e-02, 5.03386700e-02, 5.33613485e-02,
        5.88353309e-02, 6.04393973e-02, 6.16420966e-02, 6.67420256e-02,
        7.48079803e-02, 7.69484035e-02, 7.88715400e-02, 8.37697958e-02,
        8.44684984e-02, 8.63596008e-02, 8.97960182e-02, 9.02125326e-02,
        9.47191922e-02, 1.02865820e-01, 1.06544473e-01, 1.07766203e-01,
        1.13946725e-01, 1.20348159e-01, 1.21339999e-01, 1.23038545e-01,
        1.30438652e-01, 1.30840097e-01, 1.35566538e-01, 1.44103733e-01,
        1.56179252e-01, 1.65135625e-01, 1.65426897e-01, 1.68692953e-01,
        1.68741857e-01, 1.75747284e-01, 1.76036885e-01, 1.76198762e-01,
        1.77784285e-01, 1.86917568e-01, 1.87473232e-01, 1.96414918e-01,
        2.02820956e-01, 2.06614962e-01, 2.23050403e-01, 2.34180055e-01,
        2.35804972e-01, 2.37327580e-01, 2.39035769e-01, 2.456848

And now, `y`.

In [80]:
y, y.shape

(array([ 1.79779251e+00,  1.93954623e+00,  2.01114667e+00,  1.85620075e+00,
         1.85939238e+00,  1.79282884e+00,  1.95913807e+00,  1.83159670e+00,
         1.90457897e+00,  1.63782598e+00,  1.65815350e+00,  1.53237749e+00,
         1.49591128e+00,  1.66447225e+00,  1.66701509e+00,  1.49338690e+00,
         1.27025724e+00,  1.23585917e+00,  1.29871702e+00,  1.11704708e+00,
         1.16426036e+00,  1.09524804e+00,  1.13141858e+00,  1.17586044e+00,
         8.91102289e-01,  7.77249135e-01,  7.36641706e-01,  7.24680579e-01,
         5.91493888e-01,  6.14326075e-01,  4.64669932e-01,  4.25578655e-01,
         3.53330441e-01,  3.64524105e-01,  3.02168696e-01,  4.71704893e-02,
        -1.37709223e-01, -1.36886506e-01, -7.46148374e-02, -2.16799952e-01,
        -1.03420975e-01, -3.99513865e-02, -3.14948493e-01, -2.83967334e-01,
        -3.15117677e-01, -2.88210193e-01, -3.02828763e-01, -3.79098704e-01,
        -5.15705808e-01, -4.16340046e-01, -3.68158335e-01, -4.82430574e-01,
        -2.5

### Reshaping Arrays 

In [81]:
X = X.reshape(X.shape[0], 1)
X.shape

(500, 1)

In [82]:
y = y.reshape(y.shape[0], 1)
y.shape

(500, 1)

## Calculating Predictions

Defining a function to calculate the diagonal weight matrix. This function takes in the test point, the training data and the value of `tau` which corresponds to the radius of the circle surrounding a point. All the points laying in the circle are considered for the regression problem. 

In [83]:
# Weight Matrix in code. It is a diagonal matrix.def wm(point, X, tau): 
def calculateWeightMatrix(point, X, tau):
    '''
    The parameters of this function are,
		tau --> bandwidth
		X --> Training data.
		point --> the x where we want to make the prediction.
    ''' 
    
    # m is the No of training examples .
    m = X.shape[0] 
    
    # Initialising W as an identity matrix.
    w = np.mat(np.eye(m)) 
    
    # Calculating weights for all training examples [x(i)'s].
    for i in range(m): 
        xi = X[i] 
        d = (-2 * tau * tau) 
        w[i, i] = np.exp(np.dot((xi - point), (xi - point).T) / d) 
        
    return w

Now, defining a function to predict the `y` value for a point which uses the previously defined function to calculate the weight matrix.

In [84]:
def predict(X, y, point, tau):
    # m = number of training examples.
    m = X.shape[0]

    onesColumn = np.ones(m).reshape(m, 1)

    # Appending a column of ones in X to add the bias term. Just one parameter: theta, that's why adding a column of ones to X and also adding a 1 for the point where we want to predict.
    X_ = np.append(X, onesColumn, axis=1)

    # point is the x where we want to make the prediction.
    point_ = np.array([point, 1])

    # Calculating the weight matrix using the wm function we wrote      #  # earlier.
    w = calculateWeightMatrix(point_, X_, tau)

    # Calculating parameter theta using the formula.
    theta = np.linalg.pinv(X_.T * (w * X_)) * (X_.T * (w * y))

    # Calculating predictions.
    pred = np.dot(point_, theta)

    # Returning the theta and predictions
    return theta, pred

In [85]:
def getPredictionsForSingleTau(numberPredictions, tau):
    # Empty list for storing predictions.
    predictions = []

    # Predicting for all numberPredictions values and storing them in predictions.
    for point in X:
        _, predicted = predict(X, y, point, tau)
        predictions.append(predicted)

    # Reshaping predictions
    predictions = np.array(predictions).reshape(numberPredictions, 1)

    return predictions

## Plotting Predictions

In [86]:
def plotPredictions(XTest, predictionList, tauList):
    plt.figure(figsize=(20, 15), dpi=80)

    plt.plot(X, y, "o", color='orange', alpha=0.3, label='Training Data Points')

    for i, prediction in enumerate(predictionList):
        plt.plot(XTest, prediction, label=f'tau = {tauList[i]}')

    plt.legend()
    plt.show()

## Putting it all Together

In [87]:
tauList = np.arange(0, 0.1, step=0.05)
tau = 0.08

predictionList = []

# for tau in tauList:
#     predictions = getPredictionsForSingleTau(numberSamples, tau)
#     predictionList.append(predictions)

predictionList = getPredictionsForSingleTau(numberSamples, tau)

predictionList = np.array(predictionList)

plotPredictions(X, predictionList, tau)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

# References
1. [Medium Article](https://towardsdatascience.com/locally-weighted-linear-regression-in-python-3d324108efbf)
2. [Creating Random Non-Linear Data](https://www.oreilly.com/library/view/effective-amazon-machine/9781785883231/586af32b-cd7f-40f4-bfe1-bea45b67a804.xhtml)